2023-10-18 Due to the fact that I am prototyping a lot of code and changing approaches (multiindex column default to long defaul primarily), literate progrmaming style is failing, re. version one of this notebook is somewhat unsalvagable - a testament to why rushing always fails in the end. At some point I'll have to go through and fix it. in the meantime, copy code over and push ON.

In [ ]:
# set up environment

%reload_ext autoreload
%autoreload 2

from dtwalign import dtw
import pandas as pd
from wine_analysis_hplc_uv import definitions
import seaborn as sns
import seaborn.objects as so
from wine_analysis_hplc_uv.notebooks.dtw_methods import DTWNotebookMethods
from wine_analysis_hplc_uv.signal_processing.mindex_signal_processing import (
    SignalProcessor,
)
from pybaselines import Baseline
import seaborn as sns
sns.set_theme(rc={'figure.dpi': 100})
import matplotlib.pyplot as plt
from wine_analysis_hplc_uv.notebooks import eda_by_category_methods
plotter = eda_by_category_methods.Plotting()

scipro = SignalProcessor()

data = pd.read_parquet(definitions.RAW_PARQ_PATH)
data.head()

As we can see, no effect. The smoothing necessary to remove those detected peaks will result in unsatisfactory loss of signal information. Ergo better to use constraints in the peak detection algo. Also, remaining in simple long form with no multiindex massively reduces reshaping overhead and makes UDF functions much simpler to define..

Now lets add kwargs for peak finder.. added.

In [ ]:
# redo process pipe with transform rather than apply

# time std and drop vars level
pdata = (
    data.dropna()
    .pipe(scipro.standardize_time)
    .pipe(lambda df: df.set_axis(axis=0, labels=df.index.total_seconds() / 60))
    .droplevel("vars", axis=1)
)
pdata.head()

In [ ]:
# smooth
from scipy import signal

qdata = (
    pdata.melt(ignore_index=False, value_name="signal").reset_index()
    # smooth signal with window length 5 and polynomial order 2
    .assign(
        smooth_signal=lambda df: df.groupby("samplecode")["signal"].transform(
            func=signal.savgol_filter, **dict(window_length=5, polyorder=2)
        )
    )
)

qdata.head()

In [ ]:
# add bline


def bline(x, y, i):
    return pd.Series(Baseline(x, assume_sorted=True).asls(y)[0], index=i)


rdata = qdata.assign(
    bline=qdata.groupby("samplecode")["smooth_signal"].transform(
        lambda x: Baseline(x.index).asls(x)[0]
    )
).eval("bcorr=smooth_signal-bline")
rdata.head()

In [ ]:
# plot

(
    rdata.melt(
        id_vars=["samplecode", "wine", "mins"], var_name="siglabel", value_name="sig"
    ).pipe(
        lambda df: so.Plot(data=df, x="mins", y="sig", color="siglabel")
        .facet(col="samplecode", wrap=2)
        .add(mark=so.Line())
    )
)

In [ ]:
# find peaks
def peak_finder(x, kwargs: dict = dict()) -> pd.Series:
    a = signal.find_peaks(x, **kwargs)[0]
    peaks = x.iloc[a]
    return peaks


sdata = rdata.assign(
    bcorr_peaks=lambda df: df.groupby("samplecode")["bcorr"].transform(peak_finder)
)
sdata.head()

In [ ]:
# plot to display peaks
(
    sdata.melt(
        id_vars=["samplecode", "wine", "mins"], var_name="siglabel", value_name="sig"
    )
    .loc[lambda df: (df.mins < 5) & (df.mins > 3)]
    .pivot_table(index=["samplecode", "wine", "mins"], columns="siglabel", values="sig")
    .reset_index()
    .pipe(
        lambda df: so.Plot(data=df, x="mins", color="samplecode")
        .add(so.Line(), y="bcorr")
        .add(so.Dot(), y="bcorr_peaks")
    )
)

For testing purposes, we're only interested in the top 10 peaks

In [ ]:
# get top 10 peaks per sample
tdata = sdata.assign(
    bcorr_top_10=lambda df: df.groupby("samplecode", group_keys=False)[
        "bcorr_peaks"
    ].nlargest(10)
)

# plot the peaks on top of the curves
display(
    tdata.loc[lambda df: df.mins < 21].pipe(
        lambda df: (
            so.Plot(df, x="mins", color="samplecode")
            .add(so.Line(), y="bcorr")
            .add(so.Dot(), y="bcorr_top_10")
            .plot()
        )
    )
)

# peak table
(
    tdata.pipe(
        lambda df: (
            df
            if display(
                df.loc[:, ["samplecode", "wine", "mins", "bcorr_top_10"]]
                .dropna()
                .assign(n_peak=lambda df: df.groupby("samplecode").cumcount())
                .pivot(
                    columns=["samplecode", "wine"],
                    index=["n_peak"],
                    values=["mins", "bcorr_top_10"],
                )
                .reorder_levels([1, 2, 0], axis=1)
                .sort_index(axis=1)
            )
            else df
        )
    )
)

Now align, get the top 10 peaks after alignment and compare them.

In [ ]:
tdata.head()

In [ ]:
# apply dtw

# find reference

reference = (
    tdata.pivot_table(columns=["samplecode", "wine"], index=["mins"], values="signal")
    .corr()
    .mean()
    .loc[lambda df: df == df.max()]
    .pipe(lambda df: df if display(df) else df)  # display df
    .index
)

display(reference)

Notes: Use transform with a custom function if you are expecting a result with the same shape as the input i.e. a column of the dataframe. Apply rarely gets this right.

In [ ]:
def applydtw(query, ref, kwargs: dict = dict()):
    """
    apply dtw
    """
    # subset signal by warping path
    # reset index to enable reindexing after Apply functions (i.e. transform), otherwise get duplicate index error
    aligned_query = query.iloc[dtw(x=query, y=ref, **kwargs).get_warping_path()].reset_index(drop=True).values
    
    aligned_s = pd.Series(aligned_query, index=query.index)
    
    import pandas.testing as pt
    def assert_series_not_equal(*args, **kwargs):
        try:
            pt.assert_series_equal(*args, **kwargs)
        except AssertionError:
            # frames are not equal
            pass
        else:
            # frames are equal
            raise AssertionError
        return aligned_query
    
    assert_series_not_equal(left=query,right=aligned_s)
    
    return aligned_query
    

In [ ]:
dtw_kwargs = dict(window_type="sakoechiba", window_size=10)

ref_signal = (
    tdata.set_index(["samplecode", "wine"]).loc[reference].bcorr.reset_index(drop=True)
)

udata = tdata.assign(
    align=lambda x: x.groupby(["samplecode"])["bcorr"].transform(
        applydtw, ref_signal, dtw_kwargs
    )
)

udata.head()

In [ ]:
# plot alignment
(
    udata.pipe(
        lambda df: so.Plot(data=df, x="mins").add(
            so.Line(), y="align", color='samplecode'
        )
    )
)

In [ ]:
udata.set_index(["samplecode", "wine"]).loc[reference, "bcorr"].reset_index(drop=True)

In [ ]:
# torbreck-struie didnt align well. Try again?


"""
Transform is a finnicky bitch. It requires that the output of UDFs are either pd.Series or pd.DataFrames, and that the output has the same index as the input. The reason for this is that it reindexes the output onto the source DF. You would expect it to handle that logic internally, but no, it does not.

Also, avoid using multiindex as much as possible, it encourages practices which dont align with other OLAP engines such as SQL. Long over wide.

Note: You can sub in 'display()' basically anywhere to display the target before an Exception is raised
"""

(
    udata
    # get torbreck-struie and the reference signal
    .loc[
        lambda df: (
            (df.samplecode == "torbreck-struie")
            | (df.samplecode == reference.get_level_values("samplecode")[0])
        )
    ]
    # add a column containing the warped path indices
    .assign(
        wp=lambda df: df.groupby('samplecode')["bcorr"].transform(
            lambda query: pd.Series(
                dtw(
                    x=query,
                    y=df.set_index(["samplecode", "wine"])
                    .loc[reference, "bcorr"]
                    .reset_index(drop=True),
                ).get_warping_path(),
                index=query.index,
            )
        )
    )
    .reset_index(names="i")
    .assign(warped_signal=lambda df: df.bcorr.iloc[df.wp.values].reset_index(drop=True))
    .set_index("i")
    # plot
    .pipe(
        lambda df: so.Plot(data=df, x="mins").add(
            so.Line(), y="align", color='samplecode'
        )
    )
)